# Fitting Deformable Mesh Models to Raw Point Clouds

Raw observations from depth cameras are often obtained in the format of point clouds and without any information on how the points are connected. Reconstructing the surface information from point clouds (meaning building a mesh, where the list of faces is defined) is usually a standard step in 3D data processing pipelines. 

Though many approaches exist to reconstruct 3D surface from points clouds, we will explore fitting deformable mesh models for point clouds: it is a practical and simple baseline. Also, it is based on PyTorch optimization and implemented using PyTorch3D functions.

In [1]:
ply_file = 'data/pedestrian.ply' # file containing input point cloud

## Visualizing mesh with Open3d

In [17]:
import open3d

pcd = open3d.io.read_point_cloud(ply_file)
open3d.visualization.draw_geometries([pcd],mesh_show_wireframe = True)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


## Mesh fitting with PyTorch3D

In [3]:
# Importing Libraries and Packages
import os
import sys
import torch
import numpy as np

from pytorch3d.io import load_ply, save_ply
from pytorch3d.io import load_obj, save_obj
from pytorch3d.structures import Meshes
from pytorch3d.utils import ico_sphere
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import (
    chamfer_distance,
    mesh_edge_loss,
    mesh_laplacian_smoothing,
    mesh_normal_consistency,
)

In [4]:
# Defining PyTorch device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be more slowly")

In [5]:
# Loading point cloud
verts, faces = load_ply(ply_file)
verts = verts.to(device)
faces = faces.to(device) # faces is actually an empty PyTorch tensor at this point

In [6]:
# Running normalization and changing the tensor shapes for later processing
center = verts.mean(0)
verts = verts - center
scale = max(verts.abs().max(0)[0])
verts = verts / scale
verts = verts[None, :, :]

In [7]:
# Defining optimization variable 
src_mesh = ico_sphere(4, device) # it will start as a sphere and then be optimized to fit the point cloud

In [8]:
src_vert = src_mesh.verts_list()
deform_verts = torch.full(src_vert[0].shape, 0.0, device=device, requires_grad=True) # tensor of vertex displacements

In [9]:
# Defining an SGD optimizer with "deform_verts" as the optimization variable
optimizer = torch.optim.SGD([deform_verts], lr=1.0, momentum=0.9)

In [10]:
# Defining a batch of weights for different loss functions

# Weight for the primary loss
w_chamfer = 1.0 

# weights for the regularization loss functions
w_edge = 1.0
w_normal = 0.01
w_laplacian = 0.1

In [13]:
# Iterating 2,000 times for computing the loss function, computing the gradients, 
# and going along the gradient descent directions.

for i in range(0, 2000):
    if i%200==0:
        print("i = ", i)

    # Initialize optimizer
    optimizer.zero_grad()

    # Deform the mesh
    new_src_mesh = src_mesh.offset_verts(deform_verts)

    # We sample 5k points from the surface of each mesh
    # sample_trg = sample_points_from_meshes(trg_mesh, 5000)
    sample_trg = verts
    sample_src = sample_points_from_meshes(new_src_mesh, verts.shape[1])

    # We compare the two sets of pointclouds by computing (a) the chamfer loss
    loss_chamfer, _ = chamfer_distance(sample_trg, sample_src)

    # and (b) the edge length of the predicted mesh
    loss_edge = mesh_edge_loss(new_src_mesh)

    # mesh normal consistency
    loss_normal = mesh_normal_consistency(new_src_mesh)

    # mesh laplacian smoothing
    loss_laplacian = mesh_laplacian_smoothing(new_src_mesh, method="uniform")

    # Weighted sum of the losses
    loss = (
        loss_chamfer * w_chamfer
        + loss_edge * w_edge
        + loss_normal * w_normal
        + loss_laplacian * w_laplacian
    )

    # Optimization step
    loss.backward()
    optimizer.step()

i =  0
i =  200
i =  400
i =  600
i =  800
i =  1000
i =  1200
i =  1400
i =  1600
i =  1800


In [14]:
# Extracting the obtained vertices and faces
final_verts, final_faces = new_src_mesh.get_mesh_verts_faces(0)

# Resuming the original center location and scale
final_verts = final_verts * scale + center

In [15]:
# Storing the predicted mesh using save_obj
final_obj = os.path.join("output", "deform1.ply")
save_ply(final_obj, final_verts, final_faces, ascii=True)

## Visualizing the obtained mesh

In [18]:
obtained_file = os.path.join("output", "deform1.ply")

obtained_obj = open3d.io.read_point_cloud(obtained_file)
open3d.visualization.draw_geometries([obtained_obj],
                                     mesh_show_wireframe = True)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


Compared to the original input point cloud from 'pedestrian.ply', the built mesh model contains far more points (2,500 compared with 239)